## Classification de texte avec embeddings et réseau récurrent

Ce notebook présente un exemple de classification de texte avec un réseau récurrent. On utilise l'exemple de la classification de question, vu à la 3e semaine sur la classification de texte. Par exemple, la question Where is John Wayne airport ? est de type LOCATION.

Les principales composantes de ce réseau sont:

* des plongements de mots provenant de Spacy
* l'encodage des questions avec un réseau LSTM bidirectionnel (voir notes de cours).
* une couche linéaire en sortie pour faire la classification, c.-à-d. déterminer le type de question.

### 1. Création des jeux de données d'entraînement et de validation

On monte ici les données d'entraînement (disponible sur le site du cours dans la section "classification de texte). Dans cet exemple, les données sont partionnées 80%-20%, ce dernier ensemble étant utilisé pour déterminer l'époque (epoch) qui obtient le meilleur modèle.

In [2]:
train_dataset_path = "./data_rnn/questions-t3.txt"
from sklearn.model_selection import train_test_split

def load_dataset(filename):
    with open(filename) as f:
        lines = f.read().splitlines()
        labels, questions = zip(*[tuple(s.split(' ', 1)) for s in lines])
    return questions, labels

questions, labels = load_dataset(train_dataset_path)

X_train, X_valid, y_train, y_valid = train_test_split(questions, labels, test_size=0.2, shuffle=True,random_state=42)

# On converti les label textuels en index numérique
id2lable = {label_id:value for label_id, value in enumerate(list(set(labels)))}
label2id = {value:label_id for label_id, value in id2lable.items()}

y_train = [label2id[label] for label in y_train]
y_valid = [label2id[label] for label in y_valid]

nb_class = len(id2lable)


## 2. Gestion du vocabulaire et de la vectorisation des mots

En utilisant uniquement les mots contenus dans l'ensemble d'entrainement, on peut construire le vocabulaire de notre corpus. Spacy sera utilisé pour faire la tokénisation des mots ainsi que pour leur attribuer un plongement (embedding - word.vector).

Lors du test, on consultera le vocabulaire pour voir si le mot a été vu à l'entraînement. Si un mot n'a pas été vu, on le considère comme un mot inconnu .

Pour gérer le vocabulaire et les embeddings de mots, on construit des tables de correspondance permettant de : 1 - obtenir l'index d'un mot (afin de convertir les questions en séquence d'index) 2 - obtenir son embedding à partir de l'index d'un mot

In [4]:
import spacy
import numpy as np
nlp = spacy.load('en_core_web_lg')
embedding_size = nlp.meta['vectors']['width']



In [5]:
word2id = {}
id2embedding = {}

word2id[1] = "<unk>"

id2embedding[1] = np.zeros(embedding_size, dtype=np.float64)

word_index = 2

for question in X_train:
    for word in nlp(question):
        if word.text not in word2id.keys():
            word2id[word.text] = word_index
            id2embedding[word_index] = word.vector
            word_index += 1


On crée ici la classe TokenisedDataset qui sera utilisée par les dataloader pour gérer les textes durant l'entraînement du modèle.



In [6]:
import torch

from torch import LongTensor
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset
from typing import List, Dict, Tuple

class TokenisedDataset(Dataset):
    
    def __init__(self, dataset: List[str] , target: np.array, word2id: Dict[str, int], nlp_model):
        self.tokenized_dataset = [None for _ in range(len(dataset))]
        self.dataset = dataset
        self.target = target
        self.word2id = word2id
        self.nlp_model = nlp_model
    
    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        if self.tokenized_dataset[index] is None:
            self.tokenized_dataset[index] = self.tokenize(self.dataset[index])
        # print(self.tokenized_dataset[index])
        # print([self.target[index]])
        
        return LongTensor(self.tokenized_dataset[index]), LongTensor([self.target[index]]).squeeze(0)

    def tokenize(self, sentence):
        return [ self.word2id.get(word.text, 1) for word in self.nlp_model(sentence)]
    
    
train_dataset = TokenisedDataset(X_train, y_train, word2id, nlp)
valid_dataset = TokenisedDataset(X_valid, y_valid, word2id, nlp)


## 3. Construction de l'architecture du réseau
L'architecture du réseau récurrent est la suivante:

* une couche en entrée qui prend les embeddings de mots de Spacy. La taille de la couche d'entrée correspond à la taille d'embedding de Spacy.
* une couche cachée récurrent qui prend en entrée un embedding de mot et l'état caché précédent. Les neurones de cette couche sont de type LSTM, une structure de neurone qui facilite la propagation d'information sur de plus longues séquences. À noter que la couche est bi-directionnelle (voir note de cours).
* une couche de classification qui donne en sortie un score pour chacune des classes (types de question).

In [7]:
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class RNNWithEmbeddingLayer(nn.Module):
    
    def __init__(self, embedding, hidden_state_size, nb_class) :
        super().__init__()
        self.embedding_layer = nn.Embedding.from_pretrained(embedding)
        embedding_size = embedding.size()[1]
        self.rnn = nn.LSTM(embedding_size, hidden_state_size, 1, bidirectional=True)        
        self.classification_layer = nn.Linear(2 * hidden_state_size, nb_class) # Une pour chaque direction
    
    def forward(self, x, x_lenghts):
        x = self.embedding_layer(x)
        x = self._handle_rnn_output(x, x_lenghts)
        x = self.classification_layer(x)
        
        return x
    
    def _handle_rnn_output(self, x, x_lenghts):
        # On "pack" les batch pour les envoyer dans le RNN
        packed_batch = pack_padded_sequence(x, x_lenghts, batch_first=True, enforce_sorted=False)
        
        # On ne conserve que le hidden state de la dernière cellule
        # full output, (last_hidden_state, last_cell_state) = ...
        _, (last_hidden_state, _) = self.rnn(packed_batch)
        
        # On remet la batch comme première dimension
        x = torch.transpose(last_hidden_state,0,1)
        
        # On concatene les vecteurs de chacune des directions du LSTM
        x = x.reshape(len(x_lenghts),-1)
                
            
        return x

    


In [8]:
def pad_batch(batch : List[Tuple[LongTensor, LongTensor]]) -> Tuple[LongTensor, LongTensor]:
    x = [x for x,y in batch]
    x_true_length = [len(x) for x,y in batch]
    y = torch.stack([y for x,y in batch], dim=0)
    
    return ((pad_sequence(x, batch_first=True), x_true_length), y)

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=pad_batch)
valid_dataloader = DataLoader(valid_dataset, batch_size=16, shuffle=True, collate_fn=pad_batch)



La prochaine section est un artefact mécanique permettant de mettre les vecteurs d'embeddings directement dans l'architecture neuronale (les valeurs des embeddings correspondent aux poids des liens de la couche). On génère la table de correspondance entre les index des mots et les embeddings dans le format attendu par PyTorch.

In [9]:
id2embedding[0] = np.zeros(embedding_size, dtype=np.float32)
embedding_layer = np.zeros((len(id2embedding), embedding_size), dtype=np.float32)
for token_index, embedding in id2embedding.items():
    embedding_layer[token_index,:] = embedding
embedding_layer = torch.from_numpy(embedding_layer)


## 4. Entraînement du modèle
Cette partie devrait vous être familière si vous avez étudié les exemples des semaines précédentes.

In [10]:
from poutyne.framework import Experiment
from poutyne import set_seeds
import numpy as np

set_seeds(42)
hidden_size = 100

model = RNNWithEmbeddingLayer(embedding_layer, hidden_size, nb_class)
experiment = Experiment('model/embeddings_rnn', 
                        model, 
                        optimizer = "SGD", 
                        task="classification")


In [11]:
logging = experiment.train(train_dataloader, valid_dataloader, epochs=50, disable_tensorboard=True)


Epoch: 1/50 Step: 278/278 100.00% |█████████████████████████|117.37s loss: 2.070367 acc: 21.737174 fscore_micro: 0.217372 val_loss: 1.977740 val_acc: 22.571942 val_fscore_micro: 0.225719
Epoch 1: val_acc improved from -inf to 22.57194, saving file to model/embeddings_rnn\checkpoint_epoch_1.ckpt
Epoch: 2/50 Step: 278/278 100.00% |█████████████████████████|24.80s loss: 1.960707 acc: 27.160216 fscore_micro: 0.271602 val_loss: 1.914909 val_acc: 31.564748 val_fscore_micro: 0.315647
Epoch 2: val_acc improved from 22.57194 to 31.56475, saving file to model/embeddings_rnn\checkpoint_epoch_2.ckpt
Epoch: 3/50 Step: 278/278 100.00% |█████████████████████████|28.67s loss: 1.902836 acc: 31.840684 fscore_micro: 0.318407 val_loss: 1.853014 val_acc: 32.913669 val_fscore_micro: 0.329137
Epoch 3: val_acc improved from 31.56475 to 32.91367, saving file to model/embeddings_rnn\checkpoint_epoch_3.ckpt
Epoch: 4/50 Step: 278/278 100.00% |█████████████████████████|30.70s loss: 1.830566 acc: 34.068407 fscore_m

Epoch: 22/50 Step: 278/278 100.00% |█████████████████████████|16.91s loss: 0.583319 acc: 81.413141 fscore_micro: 0.814131 val_loss: 0.627289 val_acc: 80.215827 val_fscore_micro: 0.802158
Epoch 22: val_acc improved from 78.95683 to 80.21583, saving file to model/embeddings_rnn\checkpoint_epoch_22.ckpt
Epoch: 23/50 Step: 278/278 100.00% |█████████████████████████|17.70s loss: 0.560419 acc: 82.403240 fscore_micro: 0.824032 val_loss: 0.615640 val_acc: 80.665468 val_fscore_micro: 0.806655
Epoch 23: val_acc improved from 80.21583 to 80.66547, saving file to model/embeddings_rnn\checkpoint_epoch_23.ckpt
Epoch: 24/50 Step: 278/278 100.00% |█████████████████████████|18.67s loss: 0.531619 acc: 83.303330 fscore_micro: 0.833033 val_loss: 0.593748 val_acc: 81.294964 val_fscore_micro: 0.812950
Epoch 24: val_acc improved from 80.66547 to 81.29496, saving file to model/embeddings_rnn\checkpoint_epoch_24.ckpt
Epoch: 25/50 Step: 278/278 100.00% |█████████████████████████|19.51s loss: 0.511028 acc: 83.59

Epoch: 47/50 Step: 278/278 100.00% |█████████████████████████|15.96s loss: 0.195097 acc: 95.072007 fscore_micro: 0.950720 val_loss: 0.549774 val_acc: 83.453237 val_fscore_micro: 0.834532
Epoch: 48/50 Step: 278/278 100.00% |█████████████████████████|15.80s loss: 0.183964 acc: 95.297030 fscore_micro: 0.952970 val_loss: 0.538694 val_acc: 84.622302 val_fscore_micro: 0.846223
Epoch: 49/50 Step: 278/278 100.00% |█████████████████████████|17.47s loss: 0.178176 acc: 95.117012 fscore_micro: 0.951170 val_loss: 0.750887 val_acc: 77.068345 val_fscore_micro: 0.770683
Epoch: 50/50 Step: 278/278 100.00% |█████████████████████████|18.67s loss: 0.170986 acc: 95.477048 fscore_micro: 0.954771 val_loss: 0.548540 val_acc: 84.442446 val_fscore_micro: 0.844424
Restoring model from model/embeddings_rnn\checkpoint_epoch_41.ckpt


## 5. Prédiction à l'aide du modèle


In [17]:
test_dataset_path = "./data_rnn/test-questions-t3.txt"
x_test, test_labels = load_dataset(test_dataset_path)
from numpy import argmax

def obtain_prediction(sentence, label=None):
    tokenized_sentence = [word2id.get(word.text,1) for word in nlp(sentence)]
    sentence_length = len(tokenized_sentence)
    class_score = model(LongTensor(tokenized_sentence).unsqueeze(0), LongTensor([sentence_length])).detach().numpy()
    return id2lable[argmax(class_score)]


In [18]:
def evaluate(x, y):
    prediction = obtain_prediction(x)
    print("\nQ: {}. \nPred: {}, Truth: {}".format(x, prediction, y))

for test_index in range(50, 60):
    x = x_test[test_index]
    y = test_labels[test_index]
    evaluate(x, y)



Q: What is the capital of Yugoslavia ?. 
Pred: LOCATION, Truth: LOCATION

Q: Where is Milan ?. 
Pred: LOCATION, Truth: LOCATION

Q: What is the speed hummingbirds fly ?. 
Pred: QUANTITY, Truth: QUANTITY

Q: What is the oldest city in the United States ?. 
Pred: LOCATION, Truth: LOCATION

Q: What was W.C. Fields ' real name ?. 
Pred: PERSON, Truth: PERSON

Q: What river flows between Fargo , North Dakota and Moorhead , Minnesota ?. 
Pred: LOCATION, Truth: LOCATION

Q: What state did the Battle of Bighorn take place in ?. 
Pred: LOCATION, Truth: LOCATION

Q: Who was Abraham Lincoln ?. 
Pred: DEFINITION, Truth: DEFINITION

Q: What are spider veins ?. 
Pred: DEFINITION, Truth: DEFINITION

Q: What day and month did John Lennon die ?. 
Pred: TEMPORAL, Truth: TEMPORAL


In [19]:
new_sentence = "Will Bernie Sanders ever become president"
print("Q: {}. Pred:{}".format(new_sentence, obtain_prediction(new_sentence)))


Q: Will Bernie Sanders ever become president. Pred:PERSON


In [20]:
new_sentence = "Who let the dogs out"
print("Q: {}. Pred:{}".format(new_sentence, obtain_prediction(new_sentence)))


Q: Who let the dogs out. Pred:PERSON
